In [ ]:
# all the different plot parameters:

distance_measures = [kl_divergence,
                     js_divergence, 
                     wasserstein_distance, 
                     hellinger_distance, 
                     bhattacharyya, 
                     histogram_correlation, 
                     histogram_intersection]

distance_measure_names = ["kl_divergence",
                          "js_divergence", 
                          "wasserstein_distance",  
                          "hellinger_distance", 
                          "bhattacharyya", 
                          "histogram_correlation", 
                          "histogram_intersection"]

distances_short = ["kl","jsd", "wsd",  "hd", "bd", "hc", "hi"]

variables = ["prune_rate_in", "prune_rate_out", "sign_rate_in", "sign_rate_out"]
layers = ["dense1", "dense2"]

In [ ]:
# open all the distances

def get_distances( d_measure_name, layer, variable):
                
    myFile = open(f'4b Distances/{layer}_{variable}_{d_measure_name}_distances.txt', 'r')
    distances = myFile.read()
    
    return distances

In [ ]:

# plotting distances
def boxplotting(distances, distance_name):
    
    distances_boxes = []
    c_names = []

    for name in distances.keys():
        distances_boxes.append(distances[name])
        c_names.append(name)
    
    fig, ax = plt.subplots()
    ax.set_ylabel(distance_name)
    bplot = ax.boxplot(distances_boxes)
    plt.xticks(range(1,len(c_names)+1), c_names, rotation=70)
    plt.show()

In [ ]:
distances = get_distances(wasserstein_distance, "dense1", "prune_rate_in")
boxplotting(distances, "wasserstein_distance")